<a href="https://colab.research.google.com/github/mralamdari/Robotics-in-Python/blob/main/Robotics_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import scipy
import sympy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#Denavite Hartnzberg
def DH(a, alpha_degree, d, theta_degree):
  theta = np.deg2rad(theta_degree)
  alpha = np.deg2rad(alpha_degree)
  dh_table = np.array([[np.cos(theta), -np.cos(alpha)*np.sin(theta), np.sin(alpha)*np.sin(theta), a*np.cos(theta)],
                      [np.sin(theta), np.cos(alpha)*np.cos(theta), -np.sin(alpha)*np.cos(theta), a*np.sin(theta)],
                      [0 ,np.sin(alpha), np.cos(alpha), d],
                      [0, 0, 0, 1]])
  return np.float16(dh_table)

In [5]:
a1 =DH(0, -90, -999, 0)
a2 =DH(0, -90, -888, 90)
a3 =DH(0, 0, -777, 0)

s=np.dot(a1, a2)
s

array([[   0.,    0.,   -1.,    0.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)

In [6]:
np.dot(s, a3)

array([[   0.,    0.,   -1.,  777.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)

#Forwarsd Synematic

0 ===> Scara

1 ===> manipulator

2 ===> cyliendrical

3 ===> stanford

In [7]:
def forward_scara(a1, a2, d3, d4, theta_1, theta_2, theta_4):
  A01 = DH(a1, 0, 0, theta_1)
  A12 = DH(a2, 180, 0, theta_2)
  A23 = DH(0, 0, d3, 0)
  A34 = DH(0, 0, d4, theta_4)

  T40 = np.matmul(np.matmul(A01, A12), np.matmul(A23,A34))
  return np.float16(T40)

In [8]:
def forward_manipulator(a1, a2, theta_1, theta_2):
  A01 = DH(a1, 0, 0, theta_1)
  A12 = DH(a2, 0, 0, theta_2)

  T20 = np.matmul(A01, A12)
  return np.float16(T20)

In [9]:
def forward_cylindrical(alpha_2, d1, d2, d3, theta_1):
  A01 = DH(0, 0, d1, theta_1)
  A12 = DH(0, alpha_2, d2, 0)
  A23 = DH(0, 0, d3, 0)

  T30 = np.matmul(np.matmul(A01, A12), A23)
  return np.float16(T30)

In [10]:
def forward_stanford(d2, d3, theta_1, theta_2):
  A01 = DH(0, -90, 0, theta_1)
  A12 = DH(0, 90, d2, theta_2)
  A23 = DH(0, 0, d3, 0)

  T30 = np.matmul(np.matmul(A01, A12), A23)
  return np.float16(T30)

In [11]:
def wrist(alpha_4, alpha_5, d6, theta_4, theta_5, theta_6):
  A34 = DH(0, -90, 0, theta_4)
  A45 = DH(0, 90, 0, theta_5)
  A56 = DH(0, 0, d6, theta_6)

  T63 = np.matmul(np.matmul(A34, A45), A56)
  return np.float16(T63)

In [12]:
def forward_robotic(point, num=0, wist=False, a1=0, a2=0, a3=0, a4=0, a5=0, a6=0, alpha_1=0, alpha_2=0, alpha_3=0, alpha_4=0, alpha_5=0, alpha_6=0,  d1=0, d2=0, d3=0, d4=0, d5=0, d6=0, theta_1=0, theta_2=0, theta_3=0, theta_4=0, theta_5=0, theta_6=0):
  
  
  point=np.array(point).reshape(-1, 1)
  if num == 0:
    T0_3 = forward_scara(a1, a2, d3, d4, theta_1, theta_2, theta_4)
  elif num == 1:  
    T0_3 = forward_manipulator(a1, a2, theta_1, theta_2)
  elif num == 2:  
    T0_3 =  forward_cylindrical(alpha_2, d1, d2, d3, theta_1)
  else:  
    T0_3 = forward_stanford(d2, d3, theta_1, theta_2)

  if wist:
    T3_6 = wrist(alpha_4, alpha_5, d6, theta_4, theta_5, theta_6)
    T = np.matmul(T0_3, T3_6)
  else:
    T = T0_3

  R = T[:-1, :-1]
  d = T[:-1, -1].reshape(-1, 1)  
  res = np.matmul(R, point) + d

  return res.T[0]

In [13]:
forward_robotic([12,23,53],
                 num=3,
                 wist=True,
                 a1=2,
                 a2=2,
                 d1=20,
                 d2=40,
                 d3=30,
                 d4=10 ,
                 theta_1=60,
                 theta_2=-180,
                 theta_4=25)

array([-57.68408203,  31.95996094, -83.        ])

#Tabdils and Rotations
R_x, R_y, R_z

T_x, T_y, T_z

In [14]:
def T_x(a):
  TX = np.array([[1, 0, 0, a],
                 [0, 1, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
  return TX

In [15]:
def T_y(b):
  TY = np.array([[1, 0, 0, 0],
                 [0, 1, 0, b],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
  return TY

In [16]:
def T_z(c):
  TZ = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, c],
                 [0, 0, 0, 1]])
  return TZ

In [17]:
def R_x(alpha):
  alpha = np.deg2rad(alpha)
  RX = np.array([[1, 0, 0, 0],
                 [0, np.cos(alpha), -np.sin(alpha), 0],
                 [0, np.sin(alpha), np.cos(alpha), 0],
                 [0, 0, 0, 1]])
  return np.float16(RX)  

In [18]:
def R_y(betha):
  betha = np.deg2rad(betha)
  RY = np.array([[np.cos(betha), 0, np.sin(betha), 0],
                 [0, 1, 0, 0],
                 [-np.sin(betha), 0, np.cos(betha), 0],
                 [0, 0, 0, 1]])
  return np.float16(RY)

In [19]:
def R_z(gamma):
  gamma = np.deg2rad(gamma)
  RZ = np.array([[np.cos(gamma), -np.sin(gamma), 0, 0],
                 [np.sin(gamma), np.cos(gamma), 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
  return np.float16(RZ)  

In [21]:
R_z(90)

array([[ 0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float16)

In [22]:
R_y(0)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [-0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float16)

In [24]:
R_z(45)

array([[ 0.707, -0.707,  0.   ,  0.   ],
       [ 0.707,  0.707,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]], dtype=float16)

In [25]:
T_x(52)

array([[ 1,  0,  0, 52],
       [ 0,  1,  0,  0],
       [ 0,  0,  1,  0],
       [ 0,  0,  0,  1]])

In [23]:
np.dot(np.dot(R_z(90),R_y(0)), R_z(45)) + T_x(52)

array([[ 0.29296875, -0.70703125,  0.        , 52.        ],
       [ 0.70703125,  0.29296875,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  2.        ]])

#Inverse Synematic

0, Scara

1, RRR

2, stanford

In [26]:
def inverse_stanford(x, y, z, a2, a3, d1=0, left=True):
  if left:
    alpha = np.arctan2(x, y)
    beta = np.arctan2(-np.sqrt(np.abs(x**2 + y**2 - d1**2)), -d1)
    theta1 = alpha + beta
  else: 
    alpha = np.arctan2(x, y)
    beta = np.arctan2(np.sqrt(np.abs(x**2 + y**2 - d1**2)), d1)
    theta1 = alpha - beta

  theta2 = np.arctan2(x**2 + y**2, z-d1)
  D3 = np.sqrt(np.abs(x**2 + y**2 + (z-d1)**2))
  return theta1, theta2, D3

In [27]:
forward_stanford(d2=1, d3=2, theta_1=30, theta_2=45)
np.arctan2(3, 1)

1.2490457723982544

In [28]:
def inverse_rrr(x, y, z, a2, a3, d1=0, left=True):
  if left:
    alpha = np.arctan2(x, y)
    beta = np.arctan2(-np.sqrt(np.abs(x**2 + y**2 - d1**2)), -d1)
    theta1 = alpha + beta
  else: 
    alpha = np.arctan2(x, y)
    beta = np.arctan2(np.sqrt(x**2 + y**2 - d1**2), d1)
    theta1 = alpha - beta

  D = (x**2 + y**2 - d1**2 + (z-d1)**2 - a2**2 - a3**2) / 2*a2*a3
  theta3 = np.arctan2(D, -np.sqrt(np.abs(1 - D**2)))
  theta2 = np.arctan2(np.sqrt(np.abs(x**2 + y**2 - d1**2)), z-d1) - np.arctan2(a2+a3*np.cos(theta3), a3*np.sin(theta3))
  return theta1, theta2, theta3

In [29]:
def inverse_scara(x, y, z, d4, a1, a2, RR):
  theta2 = np.arccos((x**2 + y**2 - a1**2-a2**2)/(2*a1*a2))
  theta1 = np.arctan2(y, x) - np.arctan2(a2*np.sin(theta2), a1 + a2*np.cos(theta2))
  D3 = -z - d4
  theta4 = theta1 + theta2 - np.arctan2(RR[0][0], RR[0][1])
  return theta1, theta2, theta4

In [30]:
def inverse_wrist(RR):
  theta5_1 =  np.arctan2(RR[2][2], np.sqrt(np.abs(1 - RR[2][2]**2)))
  theta5_2 =  np.arctan2(RR[2][2], -np.sqrt(np.abs(1 - RR[2][2]**2)))
  if theta5_1 != 0 : # none_MONFARED
    if np.sin(theta5_1)>0:
      theta4_1 = np.arctan2(RR[0][2], RR[1][2])
      theta6_1 = np.arctan2(RR[2][0], RR[2][1])
      print(f"""
      Theta4_1: {theta4_1},
      Theta5_1: {theta5_1},
      Theta6_1: {theta6_1}""")
    if np.sin(theta5_2)<0:
      theta4_2 = np.arctan2(-RR[0][2], -RR[1][2])
      theta6_2 = np.arctan2(RR[2][0], -RR[2][1])
      print(f"""
      Theta4_2: {theta4_2},
      Theta5_2: {theta5_2},
      Theta6_2: {theta6_2}""")
  else: 
    theta5 = 0
    theta4 = (np.arctan2(RR[0][0], RR[1][0])+np.arctan2(RR[0][0], -RR[1][0]))/2
    theta6 = theta4 - np.arctan2(RR[0][0], RR[1][0])
    print(f"""
      Theta4: {theta4},
      Theta5: {theta5},
      Theta6: {theta6}""")

In [31]:
def inverse_robotic(x, y, z,RR=[], num=0, wist=False,left=True, a1=0, a2=0, a3=0, a4=0, a5=0, a6=0, d1=0, d2=0, d3=0, d4=0, d5=0, d6=0):

  if num == 0:
    T1, T2, T3 = inverse_scara(x, y, z, d4, a1, a2, RR)
  elif num == 1:  
    T1, T2, T3 = inverse_rrr(x, y, z, a2, a3, d1=0, left=True)
  else:
    T1, T2, T3 = inverse_stanford(x, y, z, a2, a3, d1=0, left=True)

  if wist:
    inverse_wrist(RR)

  return [T1, T2, T3]

In [34]:
YT = np.array([[1, 1,3], [2,8,5],[4,7,9]])
inverse_robotic(x=2,
               y=3,
               z=4,
               a2=5,
               a3=5,
               a1=2,
               num=0,
               d1=1,
               RR=YT,
               wist=False)

[-1.1760052070951348, 2.498091544796509, 0.5366881743039258]

In [35]:
inverse_robotic(x=2,
               y=3,
               z=4,
               a2=5,
               a3=5,
               num=1,
               d1=4,
               left=True,
               wist=False)

[-0.982793723247329, -2.0153104341788657, -2.3561908620481042]

In [36]:
YT = np.array([[1, 1,3], [2,8,5],[4,7,9]])
inverse_robotic(x=2,
               y=3,
               z=4,
               a2=5,
               a3=5,
               num=2,
               d1=0,
               left=False,
               wist=True,
               RR=YT)


      Theta4_1: 0.5404195002705842,
      Theta5_1: 0.7885037734282083,
      Theta6_1: 0.5191461142465229


[-0.982793723247329, 1.2722973952087173, 5.385164807134504]